**在 Colab 環境下也能執行 Selenium** 

參考： https://gist.github.com/korakot/5c8e21a5af63966d80a676af0ce15067

In [ ]:
# Colab 內建的 python 版本目前(2022/04/29)是 3.7.13 
# install chromium, its driver, and selenium
!apt update
!apt install chromium-chromedriver
!pip install selenium

In [45]:
# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
#找 user-agent 的網站： https://www.whatsmyua.info/
userAgent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36";
#許多網站會檢查 user-agent
options.add_argument("user-agent={}".format(userAgent))
#不讓瀏覽器執行在前台，而是在背景執行，開發前期(或在本機執行)先不加才看的到畫面
options.add_argument('--headless')
# 在非沙盒測試環境下，可以 root 權限運行
options.add_argument('--no-sandbox')
# 不以 /dev/shm 作為暫存區(使用 /tmp)
options.add_argument('--disable-dev-shm-usage')
# 設定瀏覽器的解析度
# options.add_argument('--window-size=1920,1080')
# 關閉 GPU ，Google 文件提到需要加上這個參數以解決部份的 bug
# options.add_argument('--disable-gpu')

# open it, go to a website, and get results
wd = webdriver.Chrome(options=options)

需要使用 Selenium 進行爬蟲的網站：


1.   該網站有多道防爬蟲機制，因此使用 Selenium 來爬可以降低被擋的機會
2.   該網站的內容加載不是用換頁的，而是類似延遲加載的方式 Lazy Loading 
3.   該網站是使用前端語言框架所寫的 Single Page App (SPA)架構，內容更動都不會變動網址。
4.   需要頻繁且複雜的互動操作才能擷取到內容的網站，例：有會員機制或彈出視窗阻擋操作等機制。
5.   自動訂票、限時搶購的網站操作。
6.   需要經常填寫資料(例：生產履歷)，但該網站卻沒提供 API



In [46]:
#此網址是 Nasdaq 網站的 Tesla 即時股價，此股價在換頁時，網址並不會更動，因此適合使用 Selenium
baseUrl = 'https://www.nasdaq.com/market-activity/stocks/tsla/latest-real-time-trades'
wd.get(baseUrl)
#隱式等待：是在嘗試發現某個元素的時候，如果沒能立刻發現，就等待固定長度的時間。預設設定是0秒。
wd.implicitly_wait(5)

In [ ]:
# 傾印網頁內容，以測試網頁是否有正確(權限)讀取完整內容
# print(wd.page_source) 

In [47]:
results = wd.find_element(By.XPATH, '//table[@class="latest-real-time-trades__table"]').get_attribute('innerHTML')
print(results)


          <thead class="latest-real-time-trades__table-headings">
            <tr class="latest-real-time-trades__row latest-real-time-trades__row--headings"><th class="latest-real-time-trades__table-heading" scope="col">NLS Time (ET)</th><th class="latest-real-time-trades__table-heading" scope="col">NLS Price</th><th class="latest-real-time-trades__table-heading" scope="col">NLS Share Volume</th></tr>
          </thead>
          <tbody class="latest-real-time-trades__table-body"><tr class="latest-real-time-trades__row"><th scope="row" class="latest-real-time-trades__cell">11:35:32</th><td scope="row" class="latest-real-time-trades__cell">$ 910.14</td><td scope="row" class="latest-real-time-trades__cell">100</td></tr><tr class="latest-real-time-trades__row"><th scope="row" class="latest-real-time-trades__cell">11:35:29</th><td scope="row" class="latest-real-time-trades__cell">$ 909.755</td><td scope="row" class="latest-real-time-trades__cell">100</td></tr><tr class="latest-real-time-

In [48]:
#TODO: 點擊下一頁
# 顯示等待：等待目標元素的出現，最多等待10秒
element = WebDriverWait(wd, 10).until(expected_conditions.presence_of_element_located((By.XPATH, '//div[@class="latest-real-time-trades__data"]')))